In [1]:
import pandas as pd
import numpy as np

In [2]:
input = """Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green"""

In [3]:
#with open('.day2-input-p1.txt', 'r') as f:
#    input = f.read()

In [4]:
key = {'red': 12, 'green': 13, 'blue': 14}

In [5]:
# ugly but who cares
data = [ x[5:].split(': ') for x in input.splitlines() ]
data = { int(x): y for (x,y) in data }
for k, v in data.items():
    data[k] = [ [ (x[1], int(x[0])) for x in [ p.split(' ') for p in b.split(', ') ] ] for b in v.split('; ') ]

In [6]:
df = pd.DataFrame.from_dict(data, orient='index').stack().to_frame()
df = pd.DataFrame(list(map(dict, df[0].values)), index=df.index).fillna(0)
df.head()

blue  red  green
1 0   3.0  4.0    0.0
  1   6.0  1.0    2.0
  2   0.0  0.0    2.0
2 0   1.0  0.0    2.0
  1   4.0  1.0    3.0

In [7]:
power_sum = df.groupby(level=0).max().prod(axis=1).sum()
power_sum

2286.0

In [8]:
def impossible(row):
    return any([row[k] > key[k] for k in key.keys()])
df['impossible'] = df.apply(impossible, axis=1)
df.head()

blue  red  green  impossible
1 0   3.0  4.0    0.0       False
  1   6.0  1.0    2.0       False
  2   0.0  0.0    2.0       False
2 0   1.0  0.0    2.0       False
  1   4.0  1.0    3.0       False

In [9]:
bad = df.loc[df[df['impossible'] == True].index].index.get_level_values(0)
bad

Int64Index([3, 4], dtype='int64')

In [10]:
df.loc[df.index.get_level_values(0).difference(bad)].index.unique(0).to_series().sum()

8